<font size="6">Модели машинного обучения</font>

# Логистическая регрессия

Общее назначение регрессии состоит в анализе связи между несколькими независимыми переменными и зависимой переменной. Зависимая переменная $z$ является взвешенной суммой независимых переменных $x_1,x_2,...,x_n$ (признаков). Веса $w_1, w_2, ..., w_n$ подбираются на обучающих данных — в этом и состоит обучение модели.

$$z(x) = w_1x_1 + w_2x_2 + ... + w_nx_n$$

Переменная $z$ может принимать значения в любом диапазоне. Чтобы получить вероятность отнесения объекта к классу, нужно привести его в диапазон от 0 до 1 с помощью функции активации.

## Переход к вероятности

В случае бинарной классификации используется сигмоида. Если получившееся значение больше 0.5, то объект относится к положительному классу, иначе — к отрицательному.

$$ σ(z(x))=\frac{1}{1+e^{-z(x)}}$$

Запишем формулу сигмоиды, используя методы из библиотеки NumPy:

- `np.exp(x)`  для подсчета $e^x$
- `np.arange()` для указания диапазона возможных значений

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

x = np.arange(-10, 10, 0.5)
z = 1/(1 + np.exp(-x))

plt.plot(x, z)
plt.xlabel("x")
plt.ylabel("Sigmoid(X)")

plt.show()

Для многоклассовой классификации используется функция активации softmax.  Вероятность $i$-го класса при наличии $K$ классов рассчитывается следующим образом:

$$\text{softmax}(z_{i}) = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$$
$$i=1,...,K$$

Для каждого объекта выбирается класс с наибольшей вероятностью.

Запишем формулу для функции софтмакс. Для подсчета суммы используем метод `np.sum()`.

In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

x = np.array([2.6, 3.2, 0.5])
print(softmax(x))
print(np.sum(softmax(x)))

### Функция потерь

Необходимо определить оптимальные параметры $w_1,...,w_n$, при которых различие между предсказанными и истинными значениями будет минимально. Рассчитать величину ошибки позволяет **функция потерь**, которую необходимо минимизировать.

Мы бы хотели такую функцию, которая очень сильно штрафует за большие отступления от истинного ответа и не сильно за небольшие ошибки.

<center><img src ="https://i.ibb.co/FKsSgWL/cross-entropy.png" width="500" ></center>

Этим задача отвечает функция кросс-энтропии ($L_{CE}$ — *Cross Entropy Loss*):

$$L_{CE}(\hat y,y) = -\sum^{K}_{i=1}y_i \cdot log(\hat y_i),$$

где $i$ — номер класса, $y$ — истинный ответ, $\hat y$ — предсказанный ответ. Если истинный ответ 0, а предсказывается 1 — loss уходит в бесконечность. Однако, чем ближе мы приближаемся к 0 — тем существенно меньше штраф. В принципе, мы могли бы изобрести и другую функцию.

В случае бинарной классификации имеет $K=2$, $y=0$ или $y=1$ ($L_{BCE}$ — *Binary Cross Entropy Loss*):

$$L_{BCE}(\hat y,y) = -(y \cdot log(\hat y)+(1-y) \cdot log(1-\hat y))$$


В случае многоклассовой классификации $K>2$. Истинный ответ $y$ — вектор длины $K$, где элемент вектора $y_c=1$, если $c$ — истинный класс, остальные элементы равны $0$.

Например, $K=3$ (классы $0,1,2$). Объект относится к классу $2$. Тогда $y = (0, 0, 1)$, $c=2$, $y_2 = 1$.

Модель предсказывает вектор $\hat y$ длины $K$. Функция кросс-энтропии имеет вид:

$$L_{CE}(\hat y,y) = -\log \hat y_c$$

## Метод градиентного спуска

Задача поиска оптимальных параметров модели сводится к задаче **поиска минимума функции потерь**.

### Точки минимума и максимума функции

Как найти минимум функции в простом случае?

- Найти производную функции.
- Найти значения, при которых производная равна нулю.
- Определить знаки производной. Когда функция возрастает, то производная положительна. Когда функция убывает, то производная отрицательна.
- Определить точки минимума и максимума. Если функция возрастала и в определенной точке начала убывать, то это точка максимума. Если функция убывала и в некоторой точке начала возрастать, то это точка минимума.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L01/function.png" width="700" ></center>

$$\text{Таблица производных:}$$
\begin{array}{|с|c|c|} \hline
 f(x) \text{ (функция)} & f'(x)  \text{ (производная)}\\ \hline
с \text{ (константа)} & 0 \\ \hline
cx & c \\ \hline
x^c & cx^{c-1} \\ \hline
\end{array}

Рассмотрим на примере функции $f(x)=x^2 + x + 3$.

Она зависит от одной переменной $x$.

Найдем производную $f'(x^2 - x + 3)$.
- $f'(x^2 - x + 3) = 2x - 1$
- $f'(x^2 - x + 3) = 0$ при $x =0.5$
-  $f'(x^2 - x + 3) < 0 $ при $x < 0.5$, $f'(x^2 - x + 3) > 0 $ при $x > 0.5$
- $f(x)$ убывает при $x < 0.5$, $f(x)$ возрастает при $x > 0.5$
- $x = 0.5$ — точка минимума

In [ ]:
import numpy as np
x = np.linspace(-2, 3, 100)
y = x**2 - x + 3
plt.figure()
plt.title("$x^2 - x + 3$")
plt.plot(x, y)
plt.xticks(np.arange(-2, 3.5, step=0.5))
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.plot(0.5,2.75, marker='.')
plt.show()

### Частная производная и градиент функции

:Если функция зависит от нескольких переменных, можно говорить о частной производной, когда все остальные переменные, кроме интересующей нас, становятся константами. Производная функции $f$ от переменной $x_1$: $\large\frac{\partial f}{\partial x_1}$.

**Градиент** в математическом анализе — это вектор, указывающий на направление максимального роста функции в заданной точке. Вектор-градиент состоит из частных производных функции от каждой из ее переменных $(x_{1},...,x_{n})$:

$$ \nabla f(\vec{x}) = \begin{bmatrix}
\displaystyle\frac{\partial f}{\partial x_1}\\
\displaystyle\frac{\partial f}{\partial x_2}\\
...\\
\displaystyle\frac{\partial f}{\partial x_n}\\
\end{bmatrix}$$

Рассмотрим функцию $f(x,y,z)=2xy^3+3z^2$. Найдем градиент функции для переменных $x,y,z$.

$$
\nabla f(x, y, z)=\begin{bmatrix}
\displaystyle\frac{\partial f}{\partial x}\\
\displaystyle\frac{\partial f}{\partial y}\\
\displaystyle\frac{\partial f}{\partial z}\\
\end{bmatrix}
=\begin{bmatrix}
2y^3\\
\\
6xy^2\\
\\
6z\\
\end{bmatrix}
$$


Для минимизации ошибки нужен антиградиент, который показывает направление скорейшего убывания функции.

### Градиентный спуск и скорость обучения

**Градиентный спуск** — это некоторый алгоритм, который помогает нам итеративно шагами перемещаться по функции и в конце концов оказаться в ее минимуме. Мы не напрямую ищем точку, где производная равна нулю, а некоторыми манипуляциями передвигаемся вниз по функции. Градиентный спуск выбирает случайную точку, находит направление самого быстрого убывания функции и двигается до ближайшего минимума вдоль этого направления.

Важно настроить размер шага или **скорость обучения** $η$ — некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который не позволит ее перескочить, но в то же время такой, чтобы тот же процесс не шел слишком медленно (как на графике слева).

<img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L01/out/learning_rate_optimal_value.png">

Если на каком-то этапе разность между старой точкой (до шага) и новой снижается ниже предела, считается, что минимум найден, алгоритм завершен.

Вспомним, что функция потерь $\mathcal{L}(y, \hat{y})$ зависит от нескольких переменных: весов $\overline{w} = (w_1, \cdots, w_m)$ и сдвига $b$.

Следовательно, мы будем использовать несколько частных производных:

- частные производные функции потерь от весов $w_1,\cdots,w_m$: $\large\frac{\partial \mathcal{L}(y, \hat{y})}{\partial w_1}, \cdots, \frac{\partial \mathcal{L}(y, \hat{y})}{\partial w_m}$
- частная производная функции потерь от сдвига $b$: $\large\frac{\partial \mathcal{L}(y, \hat{y})}{\partial b}$

После подсчета частных производных нам необходимо обновить значения весов и сдвига.

$$w_i = w_i-η\frac{\partial \mathcal{L}(y, \hat{y})}{\partial w_i}$$

$$b = b-η\frac{\partial \mathcal{L}(y, \hat{y})}{\partial b}$$

## Распознавание эмоций

Рассмотрим задачу многоклассовой классификации на примере распознавания эмоций.

Распознавание эмоций ≠ анализ тональности. Анализ тональности — выявление оценки авторов по отношению к объектам, речь о которых идёт в тексте. Она может быть позитивной, негативной или нейтральной. Эмоции могут включать не только радость или грусть, но ужас, гнев, удивление, отвращение и т.п.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L01/sentiment_emotion.png" width="700" ></center>

Чаще всего при обработке текстов стоит задача распознать 6 базовых эмоций: счастье (happiness), печаль (sadness), страх (fear), гнев (anger), отвращение (disgust), удивление (surprise). Данная классификация введена Полом Экманом в книге [[book] 📚 Basic emotions](https://www.paulekman.com/wp-content/uploads/2013/07/Basic-Emotions.pdf). Утверждается, что люди всех культур испытывают их и могут распознавать в других людях. Для каждой базовой эмоции есть соответствующее, безошибочно опознаваемое выражение лица.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L01/basic_emotions.png" width="1100" ></center>

### Загрузка и подготовка данных

Будем использовать русскоязычный набор данных, представленный в работе [[paper] 🎓 Emotion classification in Russian: feature engineering and analysis](https://link.springer.com/chapter/10.1007/978-3-030-72610-2_10).

Он размечен по **5 классам эмоций**:
- радость (joy),
- печаль (sadness),
- злость (anger),
- неуверенность (uncertainty),
- нейтральность (neutrality).

Из классификации Экмана удалена категория отвращения из-за отсутствия соответствующих данных. Категории страха и удивления  объединены в одну категорию неопределенности из-за сходства способов, которыми они выражаются.

Загрузим набор данных.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/datasets/Emotion_Classification_Russian.csv

In [ ]:
import pandas as pd
emo = pd.read_csv('Emotion_Classification_Russian.csv')
emo.head()

<center><img src ="https://i.ibb.co/Jr5SprX/cleandata.png" width="700" ></center>

Вначале всех текстовых задач идёт очистка данных от "мусора": номеро встраниц и названий журналов, преобразование или удаление таблиц и картинок и прочее приведение текста в "чистый" вид.


Помимо метки класса (label) и текста (text), присутствует столбец с лемматизированными предложениями (lemmatized). Лемматизация — процесс приведения словоформы к лемме, то есть её нормальной (словарной) форме.

<center><img src ="https://i.ibb.co/wB19yJX/lemma.webp" width="500" ></center>

В русском языке это следующие морфологические формы:

- для существительных — именительный падеж, единственное число
  - кошками → кошка;
- для прилагательных — именительный падеж, единственное число, мужской род
  - красивых → красивый;
- для глаголов, причастий, деепричастий — глагол в инфинитиве (неопределённой форме) несовершенного вида
  - бежал → бежать.

Лемматизация необходима для языков с богатой морфологией, чтобы не расширять размер словаря за счет слов, у которых одинаковый смысл, но разные морфологические формы.

В рассматриваемом наборе данных лемматизация проведена с помощью морфологического анализатора [[git] 🐾 RNNmorph](https://github.com/IlyaGusev/rnnmorph). Он использует реккурентную нейронную сеть для определения части речи и морфологических признаков слова с учетом контекста.

Рассмотрим пример его применения.

In [ ]:
!pip install -q git+https://github.com/IlyaGusev/rnnmorph

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

forms = predictor.predict(["мама", "поймала", "мышь"])
print(f'Часть речи слова "мама": {forms[0].pos}')
print(f'Лемма слова "поймала": {forms[1].normal_form}')
print(f'Морфологический анализ слова "мышь": {forms[2].tag}')

Для удобства заменим словесные обозначения классов в датасете на числовые. Метки присваиваются в алфавитном порядке:
- **a**nger → 0
- **j**oy → 1
- **n**eutrality →  2
- **s**adness → 3
- **u**ncertainty → 4

In [ ]:
emo['num_label'] = emo['label'].astype('category').cat.codes
emo.head()

Узнаем количество данных в датасете и их распределение по классам эмоций.

In [ ]:
emo.shape

In [ ]:
import matplotlib.pyplot as plt
plt.pie(emo['label'].value_counts(), labels=emo['label'].unique(), autopct='%.1f%%')
plt.show()

Проведем уже знакомые операции для подготовки данных:
- запишем в отдельные переменные лемматизированные тексты `X` и метки классов `y`;
- разделим данные на обучающую и тестовую выборку;
- осуществим векторизацию обучающей и тестовой выборок.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X, y = emo['lemmatized'], emo['num_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

vect = TfidfVectorizer(min_df=0.0002, max_df=0.2)
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)
X_train_vect, X_test_vect

### Обучение и анализ модели

Обучим модель логистической регрессии (при дефолтном параметре `max_iter=100` модель не сходится; чтобы добиться сходимости алгоритма, установим большее количество итераций).

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42, max_iter=300) # model initialization
logreg.fit(X_train_vect, y_train) # trainig
y_logreg = logreg.predict(X_test_vect) # predicting labels
y_logreg

Результаты классификации можно отразить в виде матрицы ошибок.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Greens")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True emotion')
  plt.xlabel('Predicted emotion')

class_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
cm = confusion_matrix(y_test, y_logreg)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

Метрики для каждого класса, а также усредненные метрики представлены в отчете о классификации.

In [ ]:
from sklearn.metrics import classification_report
target_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
print(classification_report(y_test, y_logreg, target_names=target_names))

Для каждого класса были посчитаны свои коэффициенты регрессии. Они представляют матрицу $K \times n$, где $K$ — количество классов, $n$ — количество признаков (слов).

In [ ]:
coefficient_matrix = logreg.coef_
print(coefficient_matrix)
print(coefficient_matrix.shape)

 Каждый из коэффициентов регрессии описывает размер вклада соответствующего признака.
 - Положительный коэффициент — признак повышает вероятность принадлежности к классу, отрицательный коэффициент — признак уменьшает вероятность.
 - Большой коэффициент — признак существенно влияет на вероятность принадлежности к классу, почти нулевой коэффициент — признак имеет небольшое влияние на вероятность результата.

Выведем признаки с самыми большими коэффициентами для каждого класса и получим наиболее характерные слова для каждой эмоции.

In [ ]:
labels = ['Злость', 'Радость', 'Нейтральность', 'Печаль', 'Неуверенность']
coefficient_matrix = logreg.coef_

for i in range(coefficient_matrix.shape[0]):

    print(f"\n{labels[i]}:")

    feature_names = vect.get_feature_names_out() # word features
    order = coefficient_matrix[i].argsort() # ascending order for coefficients
    class_coefficients = coefficient_matrix[i][order][::-1][:5] # sort and extract top-5 coefficients
    feature_names = feature_names[order][::-1][:5] # words with the highest coefficients

    for feature, coefficient in zip(feature_names, class_coefficients):
      print(feature, coefficient.round(2))

<font size="6">Литература</font>

<font size="5">Инструменты:</font>

- [[doc] 🛠️ Scikit-learn](https://scikit-learn.org/stable/) — ML алгоритмы
- [[doc] 🛠️ NumPy](https://numpy.org/) — массивы и математические функции
- [[doc] 🛠️ Pandas](https://pandas.pydata.org/) — табличные данные
- [[doc] 🛠️ Matplotlib](https://matplotlib.org/) — визуализация
- [[doc] 🛠️ NLTK](https://www.nltk.org/) — токенизация, словари стоп-слов
- [[git] 🐾 RNNmorph](https://github.com/IlyaGusev/rnnmorph) — морфологический парсер для русского и английского языков
- [[doc] 🛠️ UDPipe](https://ufal.mff.cuni.cz/udpipe) — морфологический и синтаксический парсер для различных языков

<font size="5">Методы и алгоритмы:</font>

- [[wiki] 📚 Мешок слов](https://ru.wikipedia.org/wiki/Мешок_слов)
- [[wiki] 📚 TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF)
- [[wiki] 📚 Наивный байесовский классификатор](https://ru.wikipedia.org/wiki/Наивный_байесовский_классификатор)
- [[wiki] 📚 Логистическая регрессия](https://ru.wikipedia.org/wiki/Логистическая_регрессия)